In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Vipul
[nltk_data]     Kapadia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from collections import Counter
import operator
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, TimeDistributed, RepeatVector, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False) 


In [3]:
df_english = pd.read_csv(r'F:\Coursera\Projects\lstm_translator\small_vocab_en.csv', sep = '/t', names = ['english'])
df_french = pd.read_csv(r'F:\Coursera\Projects\lstm_translator\small_vocab_fr.csv', sep = '/t', names = ['french'])



<ipython-input-3-357775e45c9e>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_english = pd.read_csv(r'F:\Coursera\Projects\lstm_translator\small_vocab_en.csv', sep = '/t', names = ['english'])
<ipython-input-3-357775e45c9e>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_french = pd.read_csv(r'F:\Coursera\Projects\lstm_translator\small_vocab_fr.csv', sep = '/t', names = ['french'])


In [4]:
#concat both the dataframes
# axis = 0 will add french data below english data, axis =1 will add french data in next column

df_new = pd.concat([df_english, df_french], axis = 1)
df_new

,english,french
0,"new jersey is sometimes quiet during autumn , ...",new jersey est parfois calme pendant l' automn...
1,the united states is usually chilly during jul...,les Ã©tats-unis est gÃ©nÃ©ralement froid en ju...
2,"california is usually quiet during march , and...","california est gÃ©nÃ©ralement calme en mars , ..."
3,the united states is sometimes mild during jun...,"les Ã©tats-unis est parfois lÃ©gÃ¨re en juin ,..."
4,"your least liked fruit is the grape , but my l...","votre moins aimÃ© fruit est le raisin , mais m..."
...,...,...
137855,"france is never busy during march , and it is ...","la france est jamais occupÃ©e en mars , et il ..."
137856,"india is sometimes beautiful during spring , a...","l' inde est parfois belle au printemps , et il..."
137857,"india is never wet during summer , but it is s...","l' inde est jamais mouillÃ© pendant l' Ã©tÃ© ,..."
137858,"france is never chilly during january , but it...","la france est jamais froid en janvier , mais i..."


In [5]:
# to print number of records
#len(which df(which column))

print('total english records = {}'.format(len(df_new['english'])))
print('total french records = {}'.format(len(df_new['french'])))

total english records = 137860
total french records = 137860


In [14]:
##Task#3 : Perform data cleaning
#remove punctuations by creating a function
#re.sub is regualr expression. substitute

def remove_punc(x):
    return re.sub('[!#?,.:";]', '', x)

df_new['english'] = df_new['english'].apply(remove_punc)
df_new['french'] = df_new['french'].apply(remove_punc)


In [22]:
english_words = []
french_words = []


#create a function to add only unique words to list
#x will pass on each row.
#x.split will split the sentense in words

def get_unique_words(x, words_list):
    for word in x.split:
        if word not in words_list:
            words_list.append(word)
            

english_words = list(df_new['english'].str.split(' ', expand=True).stack().unique())
french_words = list(df_new['french'].str.split(' ', expand=True).stack().unique())        

#for word in df_new['english'].str.split(' '):
 #   if word not in english_words:
 #       english_words.append(word)

total_english = len(english_words) - 1
print(total_english)

total_french = len(french_words) - 1

print(total_french)

#df_new['english'].apply(lambda x: get_unique_words(x, english_words))
#df_new['french'].apply(lambda x: get_unique_words(x, french_words))


#total_english = len(english_words)
#print(total_english)

#total_french = len(french_words)
#print(total_french)



199
350


In [23]:
# Maximum length (number of words) per document. We will need it later for embeddings
maxlen_english = -1
for doc in df_new.english:
    tokens = nltk.word_tokenize(doc)
    if(maxlen_english < len(tokens)):
        maxlen_english = len(tokens)
print("The maximum number of words in english document = ", maxlen_english)

maxlen_french = -1
for doc in df_new.french:
    tokens = nltk.word_tokenize(doc)
    if(maxlen_french < len(tokens)):
        maxlen_french = len(tokens)
print("The maximum number of words in french document = ", maxlen_french)

The maximum number of words in english document =  15
The maximum number of words in french document =  24


In [24]:
#Tokenization of data

#tokenizer allows us to vetorize the data by converting words to sequence of integers

def tokenize_and_pad(x, maxlen):
    tokenizer = Tokenizer(char_level=False)
    tokenizer.fit_on_texts(x)
    sequence = tokenizer.texts_to_sequences(x)
    padded = pad_sequences(sequence, maxlen = maxlen, padding= 'post')
    return tokenizer, sequence, padded

x_tokernizer, x_sequences, x_padded = tokenize_and_pad(df_new.english, maxlen_english)
y_tokernizer, y_sequences, y_padded = tokenize_and_pad(df_new.french, maxlen_french)

# Total vocab size, since we added padding we add 1 to the total word count
english_vocab_size = total_english + 1
print("Complete English Vocab Size:", english_vocab_size)

# Total vocab size, since we added padding we add 1 to the total word count
french_vocab_size = total_french + 1
print("Complete French Vocab Size:", french_vocab_size)

# function to obtain the text from padded variables
def pad_to_text(padded, tokenizer):

    id_to_word = {id: word for word, id in tokenizer.word_index.items()}
    id_to_word[0] = ''

    return ' '.join([id_to_word[j] for j in padded])

Complete English Vocab Size: 200
Complete French Vocab Size: 351


In [25]:
#split train test

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_padded, y_padded, test_size=0.1)


In [26]:
#building model
#we are going to use keras and tensorflow.
#keras is very powerful. we do not have to know LSTM functioning. we can 
#just call LSTM and say we need 128 units

# Sequential Model
model = Sequential()
# embedding layer
model.add(Embedding(english_vocab_size, 256, input_length = maxlen_english, mask_zero = True))
# encoder
model.add(LSTM(256))
# decoder
# repeatvector repeats the input for the desired number of times to change
# 2D-array to 3D array. For example: (1,256) to (1,23,256)
model.add(RepeatVector(maxlen_french))
model.add(LSTM(256, return_sequences= True ))
model.add(TimeDistributed(Dense(french_vocab_size, activation ='softmax')))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 256)           51200     
_________________________________________________________________
lstm (LSTM)                  (None, 256)               525312    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 24, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 24, 256)           525312    
_________________________________________________________________
time_distributed (TimeDistri (None, 24, 351)           90207     
Total params: 1,192,031
Trainable params: 1,192,031
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# change the shape of target from 2D to 3D
y_train = np.expand_dims(y_train, axis = 2)
y_train.shape

# train the model
model.fit(x_train, y_train, batch_size=1024, validation_split= 0.1, epochs=12)

In [ ]:
# save the model
model.save("weights.h5")

In [ ]:
# function to make prediction
def prediction(x, x_tokenizer = x_tokernizer, y_tokenizer = y_tokernizer):
    predictions = model.predict(x)[0]
    id_to_word = {id: word for word, id in y_tokenizer.word_index.items()}
    id_to_word[0] = ''
    return ' '.join([id_to_word[j] for j in np.argmax(predictions,1)])

for i in range(5):

  print('Original English word - {}\n'.format(pad_to_text(x_test[i], x_tokernizer)))
  print('Original French word - {}\n'.format(pad_to_text(y_test[i], y_tokernizer)))
  print('Predicted French word - {}\n\n\n\n'.format(prediction(x_test[i:i+1])))